In [1]:
import numpy as np
import pandas as pd
import scholarly
import sqlite3
import itertools

In [2]:
conn = sqlite3.connect('./kolnet/kolDB.db')

In [3]:
c = conn.cursor()

In [38]:
# Creating the "edgelist" table

c.execute(
    '''
    CREATE TABLE edgelist (
    coauthor text, 
    author text, 
    id text)
    '''
)

OperationalError: table edgelist already exists

In [43]:
# Creating the "metrics" table

c.execute(
    '''
    CREATE TABLE metrics (
    ID text, 
    author text, 
    cited integer, 
    cited5y integer, 
    h_index integer, 
    h_index5y integer, 
    i10index integer, 
    i10index5y integer
    )
    '''
)


In [23]:
c.execute(
    '''
    CREATE TABLE background (
    ID text,
    author text,
    interests text
    )
    '''
)

In [224]:
c.execute("INSERT INTO employees VALUES ('Ryan', 'Willett', 96000)")

In [226]:
c.execute("SELECT * FROM employees WHERE last = 'Willett'")

In [227]:
c.fetchall()

[('Ryan', 'Willett', 96000)]

In [261]:
conn.commit()

In [17]:
conn.close()

In [ ]:
class Employee: 
    def __init__(self, author, authorID, coauthor): 
        pass

In [4]:
cb_search = scholarly.search_keyword("cerebellum")

In [86]:
for i in list(range(6)):
    print(next(cb_search))

{'_filled': False,
 'affiliation': 'University Professor (Emeritus), University of Southern '
                'California',
 'citedby': 47994,
 'email': '',
 'id': '0fOhJNYAAAAJ',
 'interests': ['Neuroscience',
               'Learning',
               'Memory',
               'Conditioning',
               'Cerebellum'],
 'name': 'Richard F. Thompson  (1930-2014)',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=0fOhJNYAAAAJ'}
{'_filled': False,
 'affiliation': 'University of Minnesota',
 'citedby': 34254,
 'email': '@umn.edu',
 'id': 'xRzl__wAAAAJ',
 'interests': ['neurogenetics',
               'neurodegeneration',
               'cerebellum',
               'polyglutamine',
               'ataxia'],
 'name': 'Harry T. Orr',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=xRzl__wAAAAJ'}
{'_filled': False,
 'affiliation': 'David Mahoney Professor of Neuroscience and Neurology, '
                'Columbia University',
 'c

In [5]:
cb_author = next(cb_search).fill()

In [6]:
print(cb_author)

{'_filled': True,
 'affiliation': 'University Professor (Emeritus), University of Southern '
                'California',
 'citedby': 48042,
 'citedby5y': 7628,
 'cites_per_year': {1980: 391,
                    1981: 313,
                    1982: 524,
                    1983: 395,
                    1984: 518,
                    1985: 475,
                    1986: 556,
                    1987: 757,
                    1988: 560,
                    1989: 828,
                    1990: 672,
                    1991: 748,
                    1992: 1819,
                    1993: 782,
                    1994: 1137,
                    1995: 773,
                    1996: 2056,
                    1997: 1198,
                    1998: 960,
                    1999: 1019,
                    2000: 1067,
                    2001: 1137,
                    2002: 1706,
                    2003: 1165,
                    2004: 1384,
                    2005: 1225,
                    2

In [33]:
affiliation = cb_author.affiliation
cited = cb_author.citedby
cited5y = cb_author.citedby5y
h_index = cb_author.hindex
h_index5y = cb_author.hindex5y
i10index = cb_author.i10index
i10index5y = cb_author.i10index5y

108

In [98]:
[print(cb_author.publications[i].bib['title']) for i in range(5)]

A dot-immunobinding assay for monoclonal and other antibodies
Absence epilepsy in tottering mutant mice is associated with calcium channel defects
Neuronal position in the developing brain is regulated by mouse disabled-1
Zebrin II: a polypeptide antigen expressed selectively by Purkinje cells reveals compartments in rat and fish cerebellum
Identification of concanavalin A-binding proteins after sodium dodecyl sulfate-gel electrophoresis and protein blotting


[None, None, None, None, None]

In [141]:
print(cb_author.publications[3].fill().bib['author'])

Gino Brochu and Leonard Maler and Richard Hawkes


In [48]:
"INSERT INTO metrics VALUES(" + str(record_name) + "," + str(cited)  + "," + str(cited5y) + "," + str(h_index) + "" + str(h_index5y) + "," + str(i10index) + "," + str(i10index5y) + ")"

NameError: name 'record_name' is not defined

In [21]:
def var_extract(auth): 
    
    # Collecting simple variables
    record_name = auth.name
    record_id = auth.id
    record_interests = auth.interests
    affiliation = auth.affiliation
    cited = auth.citedby
    cited5y = auth.citedby5y
    h_index = auth.hindex
    h_index5y = auth.hindex5y
    i10index = auth.i10index
    i10index5y = auth.i10index5y
    
    print("\nExtracting information for author: " + record_name + "\n")
    
    # Assembling SQL command to write data to the "metrics" database
    sql_s = "INSERT INTO metrics VALUES(\"" + str(record_id) + "\" , \"" + str(record_name) + "\" , " + str(cited)  + "," + str(cited5y) + "," + str(h_index) + "," + str(h_index5y) + "," + str(i10index) + "," + str(i10index5y) + ")"
    print(sql_s)
    c.execute(sql_s)
    conn.commit()
    
    # Assigning the co-authors listed by the user on Google Scholar
#     coauth_manual = [auth.coauthors[i].name for i in range(len(auth.coauthors))]

    cl = [coauthor_extract(auth, i) for i in range(5)] #len(auth.publications))]
    cl_flat = list(itertools.chain.from_iterable(cl)) # Flattening list of lists
    
    # Constructing the dataframe
    nodeEL = pd.DataFrame({"coauthor": cl_flat})
    nodeEL["author"] = record_name
    nodeEL["id"] = record_id
    
    # Clean up the dataframe
    # Removing connection to self (coauthor = author)
    nodeEL = nodeEL.loc[nodeEL.coauthor != nodeEL.author]
    # Removing duplicate connections/edges for each author
    
    return nodeEL

def coauthor_extract(auth, i): 
    try: 
        coauths = auth.publications[i].fill().bib["author"].split(" and ") # Make list from coauthor string
        coauths = [ca.strip() for ca in coauths] # Remove leading and trailing whitespace
        
        print("Authors for publication " + str(i))
        print("=" * 60)
        [print(ca) for ca in coauths] # Print the list of names extracted
        print("\n")
        
        return coauths
    except: 
        return("")

def generator_db(gen):
    conn = sqlite3.connect('./kolnet/kolDB.db')
#     c = conn.cursor()
    
    for i in list(range(6)):
        el = var_extract(next(gen).fill())
        # Append the current dataframe to the database
        el.to_sql('edgelist', con = conn, index=False, if_exists='append')
    
    conn.commit()
    conn.close()

In [22]:
generator_db(cb_search)


Extracting information for author: Henry Buchtel

INSERT INTO metrics VALUES("S0cdZgMAAAAJ" , "Henry Buchtel" , 4451,803,31,13,46,21)
Authors for publication 0
D Guitton
Henry A Buchtel
RM Douglas


Authors for publication 1
Giovanni Berlucchi
Henry A Buchtel


Authors for publication 2
GP Anzola
G Bertoloni
HA Buchtel
G Rizzolatti


Authors for publication 4
Henry A Buchtel
Charles M Butter



Extracting information for author: Germund Hesslow

INSERT INTO metrics VALUES("jKmWG-YAAAAJ" , "Germund Hesslow" , 4424,1827,35,25,56,41)
Authors for publication 0
Germund Hesslow


Authors for publication 1
Dan-Anders Jirenhed
Fredrik Bengtsson
Germund Hesslow


Authors for publication 2
Christopher H Yeo
Germund Hesslow


Authors for publication 3
Germund Hesslow


Authors for publication 4
Germund Hesslow



Extracting information for author: Christian Hansel

INSERT INTO metrics VALUES("bZgWnnkAAAAJ" , "Christian Hansel" , 4276,1860,32,25,46,39)
Authors for publication 1
Chris I De Zeeuw
C